# Obtención de datos meteorológicos con la API de la AEMET

Constará de varias funciones para obtener los datos históricos a partir del 22 de junio de la API (en la zona de Cangas de Onís como municipio), y luego las previsiones para los siguientes días (en la zona de montaña de los Picos de Europa). Se construirán algunos dataframe con los que podremos construir un CSV y exportarlo. 

> NOTA: Es necesario proporcionar una API KEY para acceder a la AEMET y otra para acceder a ChatGPT.

In [33]:
# Paquetes
from api_key import api_key
from api_key_chatgpt import api_chati
import requests
import json
import os
import pandas as pd
from datetime import datetime, timedelta

## Definición de constantes

In [12]:
querystring = {"api_key": api_key}
headers = {
	'cache-control': "no-cache"
}

ID_MUNICIPIO = '33012'  # Cangas de Onís
ZONA = 'peu1'  # Zona montañosa Picos de Europa
IDEMA = "1178R"  # Sotres

fecha_inicio = "2025-06-22T00:00:00UTC"
fecha_fin = "2025-06-30T00:00:00UTC"

## Obtención de valores climatológicos pasados

Esta función usa el endpoint de la AEMET para obtener los valores climatológicos por días observados en una determinada estación, dentro de un rango de días especificado. Devuelve el dataframe con las columnas de interés.

In [ ]:
def consulta_historicos(inicio: str, fin: str, estacion:str=IDEMA) -> pd.DataFrame | None:
	url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{inicio}/fechafin/{fin}/estacion/{estacion}"

	# Si la información está ya almacenada, la leemos de un fichero
	if os.path.exists("datos_aemet_historicos.json"):
		with open("datos_aemet_historicos.json") as f:
			prediccion = json.load(f)
	else:
		# Si no, hacemos la petición y guardamos la información devuelta
		response: requests.Response = requests.request("GET", url, headers=headers)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			# print(url_datos)
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()
				with open("datos_aemet_historicos.json", "w") as f:
					json.dump(prediccion, f)

	if prediccion:
		# Convertir a dataframe y poner la fecha en el índice
		df = pd.DataFrame(prediccion)
		if "fecha" in df.columns:
			df["fecha"] = pd.to_datetime(df["fecha"])

		# Eliminar columnas que no nos interesan
		df_nuevo = df.drop(columns=["indicativo", "nombre", "provincia", "altitud", "horatmin", "horatmax", "tmed", "horaracha", "horaPresMax", "horaPresMin", "horaHrMax", "horaHrMin"])
		
		return df_nuevo

In [35]:
# Cambio en la configuración de pandas para mostrar el dataframe
pd.set_option('display.max_columns', None)      # Muestra todas las columnas
pd.set_option('display.width', None)            # Usa el ancho completo del terminal o Jupyter
pd.set_option('display.expand_frame_repr', False)  # Evita que pandas divida líneas

df_pasado = consulta_historicos(fecha_inicio, fecha_fin)
print(df_pasado)

TEXTO; 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Una vez tenemos estos datos, vamos a completarlos con la información obtenida en la observación directa y en las previsiones de esos días.

> NOTA: Para predicciones con el modelo, tendremos que imputar los valores de estas columnas, estimándolos en base a la previsión ofrecida o bien buscando la previsión directamente, aunque sea en una zona cercana.

In [ ]:
if df_pasado is not None:
	df_pasado["altonubes"] = ["low", "", "mid", "", "low", "mid", "mid"]       # low/high/mid
	df_pasado["nubosidad"] = ["abundante", "","escasa", "abundante", "escasa", "escasa", "media"]     # abundante/escasa/media
	df_pasado["lluvia"] = ["posible", "", "chubascos", "chubascos", "no", "no", "no"]
	df_pasado["SUBIR"] = ["NO", "NO", "SI", "SI", "NO", "SI", "SI"]   # SI/NO  (en base a las imágenes finales)

	print(df_pasado)

ValueError: Length of values (7) does not match length of index (8)

## Obtención de previsiones en zona de montaña

Esta función devuelve un dataframe con las previsiones de los próximos 4 días (desde hoy hasta dentro de 3 días). Devuelve también un dataframe con la información.

Como la API de la AEMET en este caso devuelve algunos datos en forma de descripción textual, conviene procesar esa descripción para imputar el valor de las variables con las que trabajamos. Para ello, emplearemos la API de ChatGPT, integrando el LLM en la aplicación, para poder enviarle un prompt y evalúe el texto que le pasemos, transformando su significado en el valor que tomen las variables categóricas que manejamos. 

### Funciones para procesar descripciones con ChatGPT

In [ ]:
# Paquetes
from openai import OpenAI

#### 1. Procesamiento del texto sobre la nubosidad:

In [ ]:
def procesa_nubes(text):
	client = OpenAI(api_key=api_chati)
	prompt = f"Dado la siguiente descripción sobre la nubosidad de un lugar: '{text}'. Devuelve un JSON con los campos 'altonubes' que tome los valores 'low', 'mid', o 'high' dependiendo de la altura de las nubes; y 'nubosidad' con los valores 'escasa', 'media' o 'abundante' dependiendo de la cantidad de nubes que diga el texto que puede haber. Responde únicamente con el JSON."

	response = client.chat.completions.create(
		model = "gpt-4",
		messages = [
			{"role": "user", "content": prompt}
		]
	)

	return response.choices[0].message.content

print(procesa_nubes("Abundante nubosidad baja en el norte, que podrá reducir la visibilidad localmente. En el resto, poco nuboso o despejado, aumentando la nubosidad por la tarde con el desarrollo de nubes de evolución diurna"))

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [17]:
def consulta_mountain(zona=ZONA):
	url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/montaña/pasada/area/{zona}/dia/"
	dict_respuesta = dict()

	for i in range(4):
		response = requests.request("GET", url+str(i), headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			print(url_datos)

			
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# En base a la respuesta dada, rellenamos un JSON propio
			

In [18]:
consulta_mountain()

https://opendata.aemet.es/opendata/sh/ddf16566
https://opendata.aemet.es/opendata/sh/f886f646
https://opendata.aemet.es/opendata/sh/c8bf7fe4
https://opendata.aemet.es/opendata/sh/359a1f45
